In [48]:
import pandas as pd
import numpy as np
import os
import talib as ta
df = pd.read_pickle("data_pairs_kucoin_future_1d.pkl").loc[15,'pair']
df['date'] = df['date'].astype(str) +' '+ df['time'].astype(str)
df['date']= pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M:%S')
df.set_index('date', inplace=True)
df.drop('time', axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
display(df)
ADX_positions  =  pd.DataFrame(index=df.index,columns = ['trending_bullish','trending_bearish',
                                                         'best_positions','non_trending'])

WMA_positions  =  pd.DataFrame(index=df.index,columns = ['WMA20_BUY','WMA20_SELL','WMA60_BUY',
                                                     'WMA60_SELL','WMA100_BUY','WMA100_SELL','WMA200_BUY','WMA200_SELL'])

MACD_positions =  pd.DataFrame(index=df.index,columns = ['trending_bearish_weak_BUY','trending_bullish_weak_SELL',
                    'trending_bearish_strong_SELL','trending_bullish_strong_BUY','cross_signal_BUY','cross_signal_SELL'])

RSI_positions  =  pd.DataFrame(index=df.index,columns = ['cross_overbought_SELL','cross_oversold_BUY'])
BB_positions   =  pd.DataFrame(index=df.index,columns = ['BB_BUY','BB_SELL'])
OBV_positions  =  pd.DataFrame(index=df.index,columns = ['OBV_BUY','OBV_SELL'])

class calculate_ta():

        def __init__(self,df):
            self.data_pairs=df
        def ADX(self):
                print('\n \n ADX ')
                self.data_pairs['ADX'] = ta.ADX(self.data_pairs['high'], self.data_pairs['low'], self.data_pairs['close'], timeperiod=14)
                DIplus = ta.PLUS_DI(self.data_pairs['high'], self.data_pairs['low'], self.data_pairs['close'], timeperiod=14)
                DIminus=ta.MINUS_DI(self.data_pairs['high'], self.data_pairs['low'], self.data_pairs['close'], timeperiod=14)

        #positions

                display(DIplus)
                display(DIminus)
                ADX_positions['trending_bullish'] = (self.data_pairs['ADX']>20) \
            & (DIplus > DIminus)
            
                ADX_positions['trending_bearish']  = (self.data_pairs['ADX']>20)\
                & (DIplus < DIminus)
                
                ADX_positions['best_positions']  = (self.data_pairs['ADX']>25)\
                & (self.data_pairs['ADX'] < DIplus)\
                & (self.data_pairs['ADX']>= DIminus) \
                | (self.data_pairs['ADX']>25)\
                & (self.data_pairs['ADX'] > DIplus)\
                & (self.data_pairs['ADX'] < DIminus)
                
                ADX_positions['non_trending']  = self.data_pairs['ADX']<20
                return ADX_positions,DIplus,DIminus

                #---------------------------------------------------------------------------------------------------------------------#

        def WMA(self):
            WMA=pd.DataFrame()
            print('\n \n WMA ')
            WMA20 = ta.WMA(self.data_pairs['close'], timeperiod=20)
            WMA60 = ta.WMA(self.data_pairs['close'], timeperiod=60)
            WMA100 = ta.WMA(self.data_pairs['close'], timeperiod=100)
            WMA200 = ta.WMA(self.data_pairs['close'], timeperiod=200)


    #positions

    # WMA20
            WMA_positions['WMA20_BUY'] =  (self.data_pairs['close'] > WMA20) & (self.data_pairs['close'].shift(1) <= WMA20)
            WMA_positions['WMA20_SELL'] = (self.data_pairs['close'] < WMA20) & (self.data_pairs['close'].shift(1) >= WMA20)

    # WMA60
            WMA_positions['WMA60_BUY'] =  (self.data_pairs['close'] > WMA60) & (self.data_pairs['close'].shift(1) <= WMA60)
            WMA_positions['WMA60_SELL'] = (self.data_pairs['close'] < WMA60) & (self.data_pairs['close'].shift(1) >= WMA60)

    # WMA100
            WMA_positions['WMA100_BUY'] =  (self.data_pairs['close'] > WMA100) & (self.data_pairs['close'].shift(1) <= WMA100)
            WMA_positions['WMA100_SELL'] = (self.data_pairs['close'] < WMA100) & (self.data_pairs['close'].shift(1) >= WMA100)

    # WMA200
            WMA_positions['WMA200_BUY'] =  (self.data_pairs['close'] > WMA200) & (self.data_pairs['close'].shift(1) <= WMA200)
            WMA_positions['WMA200_SELL'] = (self.data_pairs['close'] < WMA200) & (self.data_pairs['close'].shift(1) >= WMA200)
            for j in WMA_positions.columns:
                if j.split('_')[-1]=='SELL':
                    for i in range(len(WMA_positions)):
                        if WMA_positions.loc[i,j]==True:
                            WMA_positions.loc[i,j]=2
                elif j.split('_')[-1]=='BUY':
                    for i in range(len(WMA_positions)):
                        if WMA_positions.loc[i,j]==True:
                            WMA_positions.loc[i,j]=1

            return WMA_positions

            #---------------------------------------------------------------------------------------------------------------------#
            #MACD   

        def MACD(self):
            print('\n \n MACD ')

            EMA12 = ta.EMA(self.data_pairs['close'], timeperiod=12)
            EMA26 = ta.EMA(self.data_pairs['close'], timeperiod=26)
            macd, signal, histogram = ta.MACD(self.data_pairs['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

            macd = pd.DataFrame(macd)
            signal = pd.DataFrame(signal)
            histogram = pd.DataFrame(histogram)

    #positions

    #weak signal BUY and sell
            MACD_positions['trending_bearish_weak_BUY'] =  (macd > signal) & (macd.shift(1) <= signal)&(macd <= 0)
            MACD_positions['trending_bullish_weak_SELL'] = (macd < signal) & (macd.shift(1) >= signal)&(macd >= 0)

    #strong signal BUY and sell
            MACD_positions['trending_bearish_strong_SELL'] =  (macd < signal) & (macd.shift(1) >= signal)&(macd <= 0)
            MACD_positions['trending_bullish_strong_BUY'] = (macd > signal) & (macd.shift(1) <= signal)&(macd >= 0)


    #cross with zero line
            MACD_positions['cross_signal_BUY'] = (signal > 0) & (signal.shift(1) <= 0) #| (signal<=signal.shift()*0.05)
            MACD_positions['cross_signal_SELL'] = (signal < 0) & (signal.shift(1) >= 0)
            for j in MACD_positions.columns:
                if j.split('_')[-1]=='SELL':
                    for i in range(len(MACD_positions)):
                        if MACD_positions.loc[i,j]==True:
                            MACD_positions.loc[i,j]=2
                elif j.split('_')[-1]=='BUY':
                    for i in range(len(MACD_positions)):
                        if MACD_positions.loc[i,j]==True:
                            MACD_positions.loc[i,j]=1

            return  MACD_positions

                #---------------------------------------------------------------------------------------------------------------------#

        def RSI(self):
            print('\n \n RSI ')
            RSI = ta.RSI(self.data_pairs['close'], timeperiod=14)

    #positions

            RSI_positions['cross_overbought_SELL'] = (RSI < 75) & (RSI.shift(1) >= 75)
            RSI_positions['cross_oversold_BUY'] =   (RSI > 30) & (RSI.shift(1) <= 30)
            for j in RSI_positions.columns:
                if j.split('_')[-1]=='SELL':
                    for i in range(len(RSI_positions)):
                        if RSI_positions.loc[i,j]==True:
                            RSI_positions.loc[i,j]=2
                elif j.split('_')[-1]=='BUY':
                    for i in range(len(RSI_positions)):
                        if RSI_positions.loc[i,j]==True:
                            RSI_positions.loc[i,j]=1

            return  RSI_positions

                #----------------------------------------------------------------------------------------------------------------#

        def BBANDS(self):
            print('\n \n BBANDS ')
            upper, mid, lower = ta.BBANDS(self.data_pairs['close'], nbdevup=2, nbdevdn=2, timeperiod=20)

            upper = pd.Series(upper)
            mid = pd.Series(mid)
            lower = pd.Series(lower)

            Percent_B = ((self.data_pairs['close']-lower)/(upper-lower))*100

    #positions

            BB_positions['BB_BUY'] =  Percent_B <= 0
            BB_positions['BB_SELL'] = Percent_B >= 100
            for j in BB_positions.columns:
                if j.split('_')[-1]=='SELL':
                    for i in range(len(BB_positions)):
                        if BB_positions.loc[i,j]==True:
                            BB_positions.loc[i,j]=2
                elif j.split('_')[-1]=='BUY':
                    for i in range(len(BB_positions)):
                        if BB_positions.loc[i,j]==True:
                            BB_positions.loc[i,j]=1

            return  BB_positions

                #----------------------------------------------------------------------------------------------------------------#            

        def OBV(self):
            print('\n \n OBV ')        
            OBV = ta.OBV(self.data_pairs['close'], self.data_pairs['volume'])    
            EMA60 = ta.EMA(OBV, timeperiod=60)    
            OBV_positions['OBV_BUY']  = (OBV > EMA60) & (OBV.shift(1) <= EMA60)
            OBV_positions['OBV_SELL'] = (OBV < EMA60) & (OBV.shift(1) >= EMA60)
            for j in OBV_positions.columns:
                if j.split('_')[-1]=='SELL':
                    for i in range(len(OBV_positions)):
                        if OBV_positions.loc[i,j]==True:
                            OBV_positions.loc[i,j]=2
                elif j.split('_')[-1]=='BUY':
                    for i in range(len(OBV_positions)):
                        if OBV_positions.loc[i,j]==True:
                            OBV_positions.loc[i,j]=1


            return OBV_positions

        def position(self):
            index = ['adx', 'wma', 'macd', 'rsi', 'bbands', 'obv']
            pos = pd.DataFrame(columns = ['data'],index=index)
            pos.at['adx','data'],DIplus,DIminus=self.ADX()
            pos.at['wma','data']=self.WMA()
            pos.at['macd','data']=self.MACD()
            pos.at['rsi','data']=self.RSI()
            pos.at['bbands','data']=self.BBANDS()
            pos.at['obv','data']=self.OBV()
            return pos ,DIplus,DIminus


df,m,k=calculate_ta(df).position()
sellorbuy=pd.DataFrame(index=df.index,columns=['signal0','signal1','signal2','signal3','signal4'])
p=pd.DataFrame()
for i in df.index:
    if i=='adx':
        continue
    j=0
    n=0
    while j < len(df.loc[i,'data'].columns):
        p['sellorbuy']=df.loc[i,'data'].iloc[:,j] + df.loc[i,'data'].iloc[:,j+1]
        sellorbuy.at[i,'signal%s' % (n)]=p.copy()
        j+=2
        n+=1
sellorbuy[sellorbuy.isna()]=0

for i in sellorbuy.index:
    for j in sellorbuy.columns:
        if type(sellorbuy.loc[i,j])==int:
            continue
        sellorbuy.loc[i,j][sellorbuy.loc[i,j]==0]=np.nan
        sellorbuy.loc[i,j][sellorbuy.loc[i,j]==2]=0

,open,high,low,close,volume
0,14.028,14.453,11.864,12.188,542371
1,12.221,12.831,11.076,12.735,451212
2,12.724,13.502,11.830,12.472,294450
3,12.512,14.341,12.068,12.348,358415
4,12.310,12.603,10.119,11.824,289769
...,...,...,...,...,...
395,1.255,1.389,1.204,1.350,6558388
396,1.351,1.391,1.254,1.295,4767381
397,1.295,1.484,1.268,1.445,5922804
398,1.445,1.517,1.405,1.449,4357137



 
 ADX 


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
395    12.136707
396    11.758268
397    13.534897
398    14.024321
399    18.278869
Length: 400, dtype: float64

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
395    26.241253
396    25.311137
397    23.874319
398    23.140790
399    21.443039
Length: 400, dtype: float64


 
 WMA 

 
 MACD 

 
 RSI 

 
 BBANDS 

 
 OBV 


In [76]:
df.loc['wma','data'].iloc[0:350]
# df.loc['wma','data'].replace({False: 2}, inplace=True)
df.loc['wma','data']['signal']=df.loc['wma','data']['WMA20_BUY'] + df.loc['wma','data']['WMA20_SELL']

# df.loc['wma','data']["Anyflag"] = df.loc['wma','data'][['WMA20_BUY', 'WMA60_BUY']]
# df.loc['wma','data']['WMA20_SELL'][df.loc['wma','data']['WMA20_SELL']==2]
# df.loc['wma','data']['WMA20_BUY'][df.loc['wma','data']['WMA20_BUY']==2]

In [50]:
sellorbuy.loc['macd','signal0']


,sellorbuy
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
395,1
396,1
397,NaN
398,NaN
